## DDQN in Hockey Enviroment - Evaluation

Evaluate a trained DDQN against various environments

Base DQN implementation adapted from HW7

In [ ]:
import os
import sys
from importlib import reload

import numpy as np
from tqdm.notebook import tqdm

# Adding the parent directory to the path to enable importing
root_dir = os.path.dirname(os.path.abspath("../"))
if root_dir not in sys.path:
    sys.path.append(root_dir)

import DDQN.DDQN as ddqn
from DDQN.DQN import TargetDQNAgent, DoubleDQNAgent
from DDQN.DDQN import DuelingDQNAgent, DoubleDuelingDQNAgent
from DDQN.evaluation import compare_agents, display_stats
import hockey.hockey_env as h_env

reload(h_env)
reload(ddqn)

In [6]:
def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / float(N)

## Environment & Agent Initialization

In [7]:
env = h_env.HockeyEnv(mode=h_env.Mode.NORMAL)
env.reset()

(array([-3.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  3.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  1.66536093, -0.30052638,  0.        ,
         0.        ,  0.        ,  0.        ]),
 {'winner': 0,
  'reward_closeness_to_puck': 0.0,
  'reward_touch_puck': 0.0,
  'reward_puck_direction': 0.0})

In [9]:
STATE_PATH = "../models/hockey_double-ddqn_dev/"

In [ ]:
agent_player = DoubleDuelingDQNAgent(
    env.observation_space,
    env.discrete_action_space,
    hidden_sizes=[512],
    hidden_sizes_A=[512, 512],
    hidden_sizes_V=[512, 512],
    use_torch=False
)

In [13]:
agent_player = DoubleDQNAgent(
    env.observation_space,
    env.discrete_action_space,
    hidden_sizes=[2048],
    use_torch=False
)

In [14]:
agent_player.load_state(STATE_PATH)

agent_opp_weak = h_env.BasicOpponent(weak=True)
agent_opp_strong = h_env.BasicOpponent(weak=False)

## Evaluation

### Winning Rates Against Opponents & Match Statistics

In [15]:
weak_opp_stats = compare_agents(
    agent_player, agent_opp_weak, env, num_matches=1000, tqdm=tqdm
)
strong_opp_stats = compare_agents(
    agent_player, agent_opp_strong, env, num_matches=1000, tqdm=tqdm
)

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [16]:
print("#" * 40)
print("Stats Against Weak Opponent:")
display_stats(weak_opp_stats)

print()
print("#" * 40)
print("Stats Against Strong Opponent:")
display_stats(strong_opp_stats)

########################################
Stats Against Weak Opponent:
Player Observation Mean:
  x pos player one: -2.2381921145177914
  y pos player one: -0.19128954220907346
  angle player one: 0.13489801164515505
  x vel player one: 0.32978815184664423
  y vel player one: -0.08841922112580122
  angular vel player one: -0.009208121375889033
  x player two: 2.4818190776624864
  y player two: 4.973219457139018e-05
  angle player two: -0.003300755921293253
  y vel player two: -0.002549235534172623
  y vel player two: -0.012836448530238867
  angular vel player two: -0.002419259718056065
  x pos puck: -0.19984195619656267
  y pos puck: -0.04498851492609093
  x vel puck: -0.7174554667704407
  y vel puck: 0.023864151971035353
  left player puck keep time: 0.8702209259116437
  right player puck keep time: 0.7280879496869457

Relative Std. Change in Agent Observations:
  x pos player one: -0.17624128891833704
  y pos player one: 0.5261466659488759
  angle player one: 0.7747395161671874
  x ve

### Rendered Demonstration Against Strong Opponent

In [17]:
obs_buffer = []
reward_buffer = []
obs, _ = env.reset()
obs_opp = env.obs_agent_two()

done = False
trunc = False
step = 0
while not (done or trunc):
    step += 1
    env.render()

    a1_discr = agent_player.act(obs)
    a1 = env.discrete_to_continous_action(a1_discr)
    a2 = agent_opp_strong.act(obs_opp)

    obs, r, done, trunc, info = env.step(np.hstack([a1, a2]))
    obs_buffer.append(obs)
    reward_buffer.append(r)

    obs_opp = env.obs_agent_two()

    if done or trunc:
        print(f"Episode done in {step} steps")
        winner = info["winner"]
        if winner == 0:
            print("Draw")
        else:
            print(f"Winner: " + ("Player" if winner == 1 else "Opponent"))
        break

Episode done in 109 steps
Winner: Player


In [18]:
env.close()